# Generate Complications & Death dataframe

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

## Define Custom Functions

In [2]:
def check_lists(lists):
    for i, ls in enumerate(lists):
        for i2, ls2 in enumerate(lists):
            for i3 in ls:
                if i3 not in ls2:
                    print('\n')
                    print(i3 + ': NOT FOUND IN')
                    print(ls2)
                    sys.exit()
                        
                        
def curate(df):

    try:
        df = df[df['Facility ID'] != np.nan]
        df['Facility ID'] = df['Facility ID'].values.astype(str)
        
        ids = df['Facility ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['Facility ID'] = ids2
        
    except:
        pass
    try:
        df = df[df['Facility Name'] != np.nan]
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    return df


def rename_and_fill(df):
    
    cols1 = ['Provider ID', "Measure Start Date", "Measure End Date",
             'Hospital Name', 'Address 1', 'City/Town', 'County/Parish',
             'Telephone Number',
             ]
    
    
    cols2 = ['Facility ID', "Start Date", "End Date",
             'Facility Name', 'Address', 'City', 'County Name',
             'Phone Number',
             ]
    
    for i, col in enumerate(cols1):
        if col in list(df):
            df.rename(columns={col: cols2[i]}, inplace=True)
            
            
    cols = [
            ]
    
    for col in cols:
        if col not in list(df):
            df[col] = float('NaN')
    
    l = list(df)
    l = list(set([x for x in l if l.count(x) > 1]))
    if len(l) > 0:
        print('duplicates:', l)
        sys.exit()
        
    return df


def process2(df, lists, yr, mo):
    df = rename_and_fill(df)
    df = curate(df)
    lists.append(list(df))
    df['file_month'] = [mo]* df.shape[0]
    df['file_year'] = [yr]* df.shape[0]
    df = df.reindex(sorted(df.columns), axis=1)
    return df, lists

## Load Files

In [3]:
df_list = []
lists = []

yrs = ['2023', '2023', '2023', '2023',
       '2022', '2022', '2022', '2022',
       '2021','2021','2021', '2021', '2021',
       '2020', '2020', '2020', '2020', 
       '2019', '2019', '2019', '2019', 
       '2018', '2018', '2018', '2018',
       '2017', '2017', '2017',
       '2016', '2016', '2016', '2016',
       '2015', '2015', '2015', '2015', '2015', '2015',
       '2014', '2014', '2014',
       ]

mos = ['01', '04', '07', '10',
       '01', '04', '07', '10',
       '01', '03', '04', '07', '10', 
       '10', '07', '04', '01', 
       '10', '07', '04', '03', 
       '10', '07', '05', '01',
       '10', '07', '04',
       '12', '11', '08', '05',
       '12', '10', '07', '05', '04', '01',
       '12', '10', '07',
       ]

subdirs = ['2023/hospitals_01_2023/Complications_and_Deaths-Hospital.csv', 
           '2023/hospitals_04_2023/Complications_and_Deaths-Hospital.csv',
           '2023/hospitals_07_2023/Complications_and_Deaths-Hospital.csv',
           '2023/hospitals_10_2023/Complications_and_Deaths-Hospital.csv',
           
           '2022/hospitals_01_2022/Complications_and_Deaths-Hospital.csv', 
           '2022/hospitals_04_2022/Complications_and_Deaths-Hospital.csv', 
           '2022/hospitals_07_2022/Complications_and_Deaths-Hospital.csv',
           '2022/hospitals_10_2022/Complications_and_Deaths-Hospital.csv',
           
           '2021/hospitals_01_2021/Complications_and_Deaths-Hospital.csv',
           '2021/hospitals_03_2021/Complications_and_Deaths-Hospital.csv',
           '2021/hospitals_04_2021/Complications_and_Deaths-Hospital.csv',
           '2021/hospitals_07_2021/Complications_and_Deaths-Hospital.csv',
           '2021/hospitals_10_2021/Complications_and_Deaths-Hospital.csv',
           
           '2020/hospitals_archive_10_2020/Complications_and_Deaths_Hospital.csv',
           '2020/hospitals_archive_07_2020/Complications_and_Deaths_Hospital.csv',
           '2020/HOSArchive_Revised_Flatfiles_20200422/Complications and Deaths - Hospital.csv',
           '2020/HOSArchive_Revised_Flatfiles_20200129/Complications and Deaths - Hospital.csv',
           
           '2019/HOSArchive_Revised_Flatfiles_20191030/Complications and Deaths - Hospital.csv',
           '2019/HOSArchive_Revised_Flatfiles_20190702/Complications and Deaths - Hospital.csv',
           '2019/HOSArchive_Revised_FlatFiles_20190424/Complications and Deaths - Hospital.csv',
           '2019/HOSArchive_Revised_Flatfiles_20190321/Complications and Deaths - Hospital.csv',
           
           '2018/HOSArchive_Revised_FlatFiles_20181031/Complications and Deaths - Hospital.csv',
           '2018/HOSArchive_Revised_FlatFiles_20180725/Complications and Deaths - Hospital.csv', 
           '2018/HOSArchive_Revised_FlatFiles_20180523/Complications and Deaths - Hospital.csv',
           '2018/HOSArchive_Revised_FlatFiles_20180126/Complications and Deaths - Hospital.csv',
           
           '2017/HOSArchive_Revised_FlatFiles_20171024/Complications and Deaths - Hospital.csv',
           '2017/HOSArchive_Revised_FlatFiles_20170726/Complications and Deaths - Hospital.csv', 
           '2017/HOSArchive_Revised_Flatfiles_20170428/Complications - Hospital.csv', 
           
           '2016/HOSArchive_Revised_Flatfiles_20161219/Complications - Hospital.csv',
           '2016/Hospital_Revised_FlatFiles_20161110/Complications - Hospital.csv', 
           '2016/HOSArchive_Revised_FlatFiles_20160810/Complications - Hospital.csv',
           '2016/HOSArchive_Revised_FlatFiles_20160504/Complications - Hospital.csv',
           
           '2015/HOSArchive_Revised_FlatFiles_20151210/Complications - Hospital.csv',
           '2015/HOSArchive_Revised_FlatFiles_20151008/Complications - Hospital.csv',
           '2015/HOSArchive_Revised_FlatFiles_20150716/Complications - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150506/Readmissions Complications and Deaths - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150416/Readmissions Complications and Deaths - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150122/Readmissions Complications and Deaths - Hospital.csv',
           
           '2014/HOSArchive_Revised_Flatfiles_20141218/Readmissions Complications and Deaths - Hospital.csv',
           '2014/HOSArchive_Revised_Flatfiles_20141023/Readmissions Complications and Deaths - Hospital.csv',
           '2014/HOSArchive_Revised_Flatfiles_20140717/Readmissions Complications and Deaths - Hospital.csv',
           ]

for i, subdir in enumerate(subdirs):
    with io.capture_output() as captured: df = pd.read_csv(main_dir + subdir, encoding = "ISO-8859-1")
    print(subdir + ' :  (rows, columns) =', df.shape)
    df, lists = process2(df, lists, yrs[i], mos[i])
    df_list.append(df)

check_lists(lists)
df = pd.concat(df_list)
print('df.shape:', df.shape)


df = df.filter(items=['Facility ID', 'Facility Name', 'file_month', 'file_year',
                      'Measure ID', 'Measure Name', 'Start Date', 'End Date',
                      'Score', 'Higher Estimate', 'Lower Estimate',
                      'Compared to National', 'Denominator', 'Footnote'], axis=1)

df.head()

2023/hospitals_01_2023/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92112, 18)
2023/hospitals_04_2023/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92055, 18)
2023/hospitals_07_2023/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (91922, 18)
2023/hospitals_10_2023/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (91922, 18)
2022/hospitals_01_2022/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92112, 18)
2022/hospitals_04_2022/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92112, 18)
2022/hospitals_07_2022/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92017, 18)
2022/hospitals_10_2022/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (92055, 18)
2021/hospitals_01_2021/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (91114, 18)
2021/hospitals_03_2021/Complications_and_Deaths-Hospital.csv :  (rows, columns) = (91114, 18)
2021/hospitals_04_2021/Complications_and_Deaths-Hospital.csv

,Facility ID,Facility Name,file_month,file_year,Measure ID,Measure Name,Start Date,End Date,Score,Higher Estimate,Lower Estimate,Compared to National,Denominator,Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,04/01/2018,03/31/2021,2.4,4.2,1.4,No Different Than the National Rate,102,NaN
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,MORT_30_AMI,Death rate for heart attack patients,07/01/2018,06/30/2021,12.4,15.4,9.8,No Different Than the National Rate,317,NaN
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,MORT_30_CABG,Death rate for CABG surgery patients,07/01/2018,06/30/2021,4.7,7.8,2.7,No Different Than the National Rate,172,NaN
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,MORT_30_COPD,Death rate for COPD patients,07/01/2018,06/30/2021,8.5,12.1,6.0,No Different Than the National Rate,182,NaN
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,MORT_30_HF,Death rate for heart failure patients,07/01/2018,06/30/2021,8.3,10.3,6.6,Better Than the National Rate,630,NaN


In [4]:
d = {'COMP_HIP_KNEE': 'COMP HIP KNEE',
     'MORT_30_AMI': 'MORT 30 AMI',
     'MORT_30_CABG': 'MORT 30 CABG',
     'MORT_30_COPD': 'MORT 30 COPD', 
     'MORT_30_HF': 'MORT 30 HF',
     'MORT_30_PN': 'MORT 30 PN',
     'MORT_30_STK': 'MORT 30 STK',
     'PSI_03': 'PSI 3',
     'PSI_04': 'PSI 4',
     'PSI_06': 'PSI 6',
     'PSI_08': 'PSI 8',
     'PSI_09': 'PSI 9',
     'PSI_10': 'PSI 10',
     'PSI_11': 'PSI 11',
     'PSI_12': 'PSI 12',
     'PSI_13': 'PSI 13',
     'PSI_14': 'PSI 14',
     'PSI_15': 'PSI 15',
     'PSI_90': 'PSI 90',
     'PSI_10_POST_KIDNEY': 'PSI 10',
     'PSI_11_POST_RESP': 'PSI 11',
     'PSI_12_POSTOP_PULMEMB_DVT': 'PSI 12',
     'PSI_13_POST_SEPSIS': 'PSI 13',
     'PSI_14_POSTOP_DEHIS': 'PSI 14',
     'PSI_15_ACC_LAC': 'PSI 15',
     'PSI_3_ULCER': 'PSI 3',
     'PSI_4_SURG_COMP': 'PSI 4',
     'PSI_6_IAT_PTX': 'PSI 6',
     'PSI_8_POST_HIP': 'PSI 8', 
     'PSI_90_SAFETY': 'PSI 90',
     'PSI_9_POST_HEM': 'PSI 9',
     'PSI_7_CVCBI': 'PSI 7',
     'READM_30_AMI': 'READM 30 AMI',
     'READM_30_COPD': 'READM 30 COPD',
     'READM_30_HF': 'READM 30 HF',
     'READM_30_HIP_KNEE': 'READM 30 HIP KNEE',
     'READM_30_HOSP_WIDE': 'READM 30 HOSP WIDE',
     'READM_30_PN': 'READM 30 PN',
     'READM_30_STK': 'READM 30 STK',
}

df['Measure ID'].replace(to_replace=d, inplace=True)
print(df['Measure ID'].unique())


d = {'A wound that splits open  after surgery on the abdomen or pelvis': 'Postoperative wound dehiscence rate', 
     'A wound that splits open after surgery on the abdomen or pelvis' : 'Postoperative wound dehiscence rate', 
     
     
     'Acute Myocardial Infarction (AMI) 30-Day Mortality Rate': 'AMI 30-Day Mortality Rate',
     'Death rate for heart attack patients': 'AMI 30-Day Mortality Rate', 
     
     'Death rate for CABG': 'Death rate for CABG surgery patients', 
     'Death rate for COPD patients': 'Death rate for COPD patients', 
     'Death rate for chronic obstructive pulmonary disease (COPD) patients': 'Death rate for COPD patients', 
     
     'Death rate for heart failure patients'   : 'Heart failure 30-Day Mortality Rate', 
     'Heart failure (HF) 30-Day Mortality Rate': 'Heart failure 30-Day Mortality Rate', 
     
     'Death rate for pneumonia patients'   : 'Pneumonia 30-Day Mortality Rate',  
     'Pneumonia (PN) 30-Day Mortality Rate': 'Pneumonia 30-Day Mortality Rate', 
     
     'Perioperative Hemorrhage or Hematoma Rate': 'Perioperative hemorrhage or hematoma rate', 
     'Perioperative hemorrhage or hematoma rate': 'Perioperative hemorrhage or hematoma rate',
     
     'Postoperative Acute Kidney Injury Requiring Dialysis Rate': 'Postoperative acute kidney injury requiring dialysis rate', 
     'Postoperative acute kidney injury requiring dialysis rate': 'Postoperative acute kidney injury requiring dialysis rate', 
     
     'Postoperative Respiratory Failure Rate': 'Postoperative respiratory failure rate', 
     'Postoperative respiratory failure rate': 'Postoperative respiratory failure rate', 
     
     'Serious blood clots after surgery': 'Perioperative pulmonary embolism or deep vein thrombosis rate', 
     'Blood stream infection after surgery': 'Postoperative sepsis rate', 
     
     'Accidental cuts and tears from medical treatment': 'Abdominopelvic accidental puncture or laceration rate', 
     'Deaths among Patients with Serious Treatable Complications after Surgery': 'Death rate among surgical inpatients with serious treatable complications', 
     'Pressure sores': 'Pressure ulcer rate', 
     
     'Collapsed lung due to medical treatment': 'Iatrogenic pneumothorax rate', 
     'Broken hip from a fall after surgery': 'In-hospital fall with hip fracture rate', 

     'CMS Medicare PSI 90: Patient safety and adverse events composite': 'Patient safety and adverse events composite', 
     'Serious complications': 'Patient safety and adverse events composite',

     'Rate of unplanned readmission for chronic obstructive pulmonary disease (COPD) patients': 'Rate of unplanned readmission for COPD patients',
   
     'Rate of readmission after discharge from hospital (hospital-wide)': 'Hospital-wide readmission rate after discharge from hospital',
     'Pneumonia (PN) 30-Day Readmission Rate': 'Pneumonia 30-Day Readmission Rate',
     'Heart failure (HF) 30-Day Readmission Rate': 'Heart failure 30-Day Readmission Rate',
     'Acute Myocardial Infarction (AMI) 30-Day Readmission Rate': 'Acute Myocardial Infarction 30-Day Readmission Rate',
     
    }

df['Measure Name'].replace(to_replace=d, inplace=True)
print(sorted(df['Measure Name'].unique()), '\n')


['COMP HIP KNEE' 'MORT 30 AMI' 'MORT 30 CABG' 'MORT 30 COPD' 'MORT 30 HF'
 'MORT 30 PN' 'MORT 30 STK' 'PSI 3' 'PSI 4' 'PSI 6' 'PSI 8' 'PSI 9'
 'PSI 10' 'PSI 11' 'PSI 12' 'PSI 13' 'PSI 14' 'PSI 15' 'PSI 90' 'PSI 7'
 'READM 30 AMI' 'READM 30 COPD' 'READM 30 HF' 'READM 30 HIP KNEE'
 'READM 30 HOSP WIDE' 'READM 30 PN' 'READM 30 STK']
['AMI 30-Day Mortality Rate', 'Abdominopelvic accidental puncture or laceration rate', 'Acute Myocardial Infarction 30-Day Readmission Rate', 'Death rate among surgical inpatients with serious treatable complications', 'Death rate for CABG surgery patients', 'Death rate for COPD patients', 'Death rate for stroke patients', 'Heart failure 30-Day Mortality Rate', 'Heart failure 30-Day Readmission Rate', 'Hospital-wide readmission rate after discharge from hospital', 'Iatrogenic pneumothorax rate', 'In-hospital fall with hip fracture rate', 'Infections from a large venous catheter', 'Patient safety and adverse events composite', 'Perioperative hemorrhage or hemat

In [5]:
tdf = df.copy(deep=True)
tdf['Measure'] = tdf['Measure Name'] + ' (' + tdf['Measure ID'] + ')'
tdf = tdf.drop(labels=['Higher Estimate','Lower Estimate', 'Measure ID', 'Measure Name',
                      'Compared to National','Denominator','Footnote'], axis=1)

Measures = sorted(tdf['Measure'].unique())
main_df = pd.DataFrame(columns=['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Start Date', 'End Date'])

for i, mi in enumerate(Measures):
    
    ttdf = tdf[tdf['Measure'] == mi]
    measures = sorted(ttdf['Measure'].unique())
    
    df2 = pd.DataFrame(columns=['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Start Date', 'End Date'])

    for j, m in enumerate(measures):
    
        tdf2 = ttdf[ttdf['Measure'] == m]
        tdf2['Score'] = pd.to_numeric(tdf2['Score'], errors='coerce')
        tdf2.rename(columns={'Score': m}, inplace=True)
        
        tdf2.drop(labels=['Measure'], axis=1, inplace=True)
        
        df2 = df2.merge(tdf2, on=['Facility ID', 'Facility Name', 'file_month', 'file_year',
                                 'Start Date', 'End Date'], how='outer')
    
    main_df = main_df.merge(df2, on=['Facility ID', 'Facility Name', 'file_month', 'file_year',
                                    'Start Date', 'End Date'], how='outer')

print(main_df.shape)
main_df.drop_duplicates(inplace=True)
print(main_df.shape)
main_df.drop_duplicates(subset = ['Facility ID', 'Facility Name', 'file_month','file_year', 
                                  'Start Date', 'End Date'], inplace=True)
print(main_df.shape)
main_df.head()

(582370, 34)
(582370, 34)
(582370, 34)


,Facility ID,Facility Name,file_month,file_year,Start Date,End Date,AMI 30-Day Mortality Rate (MORT 30 AMI),Abdominopelvic accidental puncture or laceration rate (PSI 15),Acute Myocardial Infarction 30-Day Readmission Rate (READM 30 AMI),Death rate among surgical inpatients with serious treatable complications (PSI 4),Death rate for CABG surgery patients (MORT 30 CABG),Death rate for COPD patients (MORT 30 COPD),Death rate for stroke patients (MORT 30 STK),Heart failure 30-Day Mortality Rate (MORT 30 HF),Heart failure 30-Day Readmission Rate (READM 30 HF),Hospital-wide readmission rate after discharge from hospital (READM 30 HOSP WIDE),Iatrogenic pneumothorax rate (PSI 6),In-hospital fall with hip fracture rate (PSI 8),Infections from a large venous catheter (PSI 7),Patient safety and adverse events composite (PSI 90),Perioperative hemorrhage or hematoma rate (PSI 9),Perioperative pulmonary embolism or deep vein thrombosis rate (PSI 12),Pneumonia 30-Day Mortality Rate (MORT 30 PN),Pneumonia 30-Day Readmission Rate (READM 30 PN),Postoperative acute kidney injury requiring dialysis rate (PSI 10),Postoperative hemorrhage or hematoma rate (PSI 9),Postoperative respiratory failure rate (PSI 11),Postoperative sepsis rate (PSI 13),Postoperative wound dehiscence rate (PSI 14),Pressure ulcer rate (PSI 3),Rate of complications for hip/knee replacement patients (COMP HIP KNEE),Rate of readmission after hip/knee surgery (READM 30 HIP KNEE),Rate of unplanned readmission for COPD patients (READM 30 COPD),Rate of unplanned readmission for stroke patients (READM 30 STK)
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,07/01/2018,06/30/2021,12.4,NaN,NaN,NaN,4.7,8.5,16.4,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010005,MARSHALL MEDICAL CENTERS,01,2023,07/01/2018,06/30/2021,12.6,NaN,NaN,NaN,NaN,8.1,16.6,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,01,2023,07/01/2018,06/30/2021,16.5,NaN,NaN,NaN,3.5,7.8,18.9,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,01,2023,07/01/2018,06/30/2021,NaN,NaN,NaN,NaN,NaN,10.3,NaN,13.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,01,2023,07/01/2018,06/30/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save dataframe

In [8]:
print(main_df.shape)
tdf = main_df.drop(labels=['Start Date', 'End Date'], axis=1)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)
tdf.to_pickle('~/GitHub/hospitals-data-archive/dataframes/partial_dataframes/complication_and_deaths_df.pkl.gz', protocol=5, compression='gzip')


(582370, 34)
(503201, 32)


## Save measurement dates

In [9]:
# Columns to keep as is
id_cols = ['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Start Date', 'End Date']

# Melt the specific columns and create the 'Measure' and 'Score' columns
measures_df = main_df.melt(id_vars=id_cols, var_name='Measure Name', value_name='Score')
measures_df.drop(labels=['Score', 'Facility ID', 'Facility Name'], axis=1, inplace=True)

print(measures_df.shape)
measures_df.drop_duplicates(inplace=True)
measures_df.reset_index(drop=True, inplace=True)
print(measures_df.shape)

measures_df['Start Date'] = pd.to_datetime(measures_df['Start Date'])
measures_df['End Date'] = pd.to_datetime(measures_df['End Date'])
measures_df.to_csv('~/GitHub/hospitals-data-archive/measure_dates/complication_and_deaths_df.csv')

measures_df.head()

(16306360, 5)
(3388, 5)


,file_month,file_year,Start Date,End Date,Measure Name
0,01,2023,2018-07-01,2021-06-30,AMI 30-Day Mortality Rate (MORT 30 AMI)
1,04,2023,2018-07-01,2021-06-30,AMI 30-Day Mortality Rate (MORT 30 AMI)
2,07,2023,2019-07-01,2022-06-30,AMI 30-Day Mortality Rate (MORT 30 AMI)
3,10,2023,2019-07-01,2022-06-30,AMI 30-Day Mortality Rate (MORT 30 AMI)
4,01,2022,2017-07-01,2019-12-01,AMI 30-Day Mortality Rate (MORT 30 AMI)
